In [136]:
import re
f = open("alldeq14.txt", "r+")
deqsstr = f.read()
#deqslist = re.split("\n", deqsstr)

In [137]:
deqslist= re.findall("deq14.b.*", deqsstr)
#deqslist = '\n'.join(deqssample)
#print(deqssample)
len(deqslist)

3516

In [138]:
from urllib import request

emailstext = ''

s3_root = "https://flint-text.s3.amazonaws.com/"
for deq in deqslist:       
    url = s3_root + deq
    response = request.urlopen(url)
    raw_text = response.read().decode('utf8')
    emailstext = "".join((raw_text, emailstext)) #changed this line

In [139]:
#print(emailstext) #see full email output

In [140]:
froms = re.findall("From:.*[A-Za-z].*", emailstext)
fromstr = '\n'.join(froms)
def refroms(fromstr):
    fromstr_orgs = re.sub("From:\W","", fromstr) #with (ORG)
    fromstr_noendspace1 = re.sub("\s+\n","\n", fromstr_orgs)
    fromstr_noendspace2 = re.sub("\s$","",fromstr_noendspace1)
    return fromstr_noendspace2

newfromstr = str(refroms(fromstr))
#print(newfromstr)

In [141]:
ons = re.findall("On.*wro[tf]e", emailstext)
onstr = '\n'.join(ons)
def reons(onstr):
    onstr1 = re.sub("On\s.*at\s[0-9]*:[0-9][0-9]\s[AP]M\W*","", onstr)
    onstr2 = re.sub("On\s.*\s[0-9]*","", onstr1)
    onstr3 = re.sub("[0-9]","",onstr2)
    onstr4 = re.sub("wrote","",onstr3)
    onstr_noendspace1 = re.sub("\s+\n","\n", onstr4)
    onstr_noendspace2 = re.sub("\s+$","", onstr_noendspace1)
    return onstr_noendspace2
    
newonstr = str(reons(onstr))
#print(newonstr)

In [142]:
roughlist = []
fromsenders = re.split('\n', newfromstr) #splits from str into list
onsenders = re.split('\n',newonstr) #splits on... str into list

#combines the two lists; roughlist contains names found in From: header and On... header
roughlist.extend(onsenders)
roughlist.extend(fromsenders)

roughstr = '\n'.join(roughlist) #makes list into string
print("rough list of " + str(len(roughlist)) + " things that might be names:")
print("- - -")
#print(roughstr)

rough list of 2141 things that might be names:
- - -


In [143]:
ultimatelist = [] #creates final list for final sender names to be put into
ultimatelist.clear()

In [144]:
# extracts sender lines formatted in [Last, First] 
lastcommafirstdude = re.findall("[A-Za-z]+,\s[A-Za-z]+",roughstr)
ultimatelist.extend(lastcommafirstdude) #puts these names into ultimatelist
#print(lastcommafirstdude)
len(lastcommafirstdude) #number of names already in LAST, FIRST format

1679

In [145]:
# extracts sender lines formatted in [First Last] 
firstlast = re.findall("[A-Z][a-z]+[^\n.,-_A-Za-z][A-Z][a-z][A-Za-z]+",roughstr)
#firstlast
#len(firstlast)

In [146]:
#problem: for "Shekter Smith, Liane", "Shekter Smith" is read as a [First Last]
#band-aid solution:
unwanted = {'Shekter Smith', 'Mary Beth','Tracy Jo'}
firstlast1 = [e for e in firstlast if e not in unwanted]
firstlaststring = '\n'.join(firstlast1)
#print(firstlaststring)

In [147]:
#LIST OF ALL NAMES (FIRST LAST FIRST LAST etc.)
allflnames = re.findall('[A-Z][a-z][A-Za-z]+',firstlaststring)
#print(allflnames)
#len(allflnames)

In [148]:
#LIST OF FIRST NAMES
firstnames = re.findall('^[A-Z][a-z]+\s|\n[A-Z][a-z]+\s',firstlaststring)
firstnames1 = ' '.join(firstnames)
firstnames2 = re.sub("\n","",firstnames1)
firstnameslist = re.split("\s+",firstnames2)
firstnameslist.pop()
#print("first names:")
#print(firstnameslist)
#print("")

#LIST OF LAST NAMES
lastnames1 = re.findall('.[A-Z][a-z]+',firstlaststring)
lastnames2 = ''.join(lastnames1)
lastnameslist = re.split('\s',lastnames2)
del lastnameslist[0]
#print("last names:")
#print(lastnameslist)
#print(" ")
#if len(lastnameslist) == len(firstnameslist):
    #print("yay, there are " + str(len(lastnameslist)) + " first and last names.")
#else:
    #print("drat! the number of first names found doesn't match the number of last names found")

In [149]:
print("Reformatted names:")
print("- - -")
for idx, value in enumerate(firstnameslist):
    key = lastnameslist[idx] + ", " + firstnameslist[idx]
    #print(key)
    ultimatelist.append(key) #adds these reformatted names to ultimatelist

Reformatted names:
- - -


In [150]:
roughlist2 = []
roughlist2.extend(lastcommafirstdude)
roughlist2.extend(firstlast1)
#roughlist2 = sum of names found to be in [First Last] and [Last, First]
            
msg = len(roughlist) - (len(roughlist2))
if msg > 0:
    n = str(len(roughlist))
    h = str(len(ultimatelist))
    print("out of " + n + " initial items, " + h + " are in ultimatelist.")


out of 2141 initial items, 2092 are in ultimatelist.


In [151]:
print("ULTIMATE LIST of " + str(len(ultimatelist)) + " SENDER NAMES in [Last, First]:")
print("- - -")
ultimateliststr = '\n'.join(ultimatelist)
#print(ultimateliststr)
# final list :o) 

ULTIMATE LIST of 2092 SENDER NAMES in [Last, First]:
- - -


In [152]:
ultimatelist_senders = list(dict.fromkeys(ultimatelist))

print(str(len(ultimatelist_senders)) + " unique senders found:")
print("")
#print(ultimatelist_senders)
ultimatelist_sendersstr = "\n".join(ultimatelist_senders)
#print(ultimatelist_sendersstr)

207 unique senders found:



In [153]:
# # # descriptives # # #

In [154]:
froms.clear()
ons.clear()
froms = re.findall("From:.*[A-Za-z].*", emailstext)
ons = re.findall("On.*wro[tf]e", emailstext)
print((str(len(ons) + len(froms))) + " total froms and ons vs. " + str(len(ultimatelist)) + " senders in ultimatelist")
percent = str(round(((len(ultimatelist))/(len(ons) + len(froms))*100),2))
print(percent + "%")

2154 total froms and ons vs. 2092 senders in ultimatelist
97.12%


In [155]:
fromsandonsnames = (len(onsenders)+len(fromsenders))
percentofons = (((len(onsenders)) / fromsandonsnames) * 100)
percentofonsr = round(percentofons,2)
print(str(percentofonsr) + "% estimated to have an On... header")

9.67% estimated to have an On... header


In [156]:
def namecounter(uniquenames,allnames):
    name_freq = []
    pair = ()
    for uname in uniquenames:
        if uname in allnames:
            number = allnames.count(uname)
            pair = (uname,number)
            name_freq.append(pair)
    return name_freq
    print(len(name_freq))
    
listname_freq = namecounter(ultimatelist_senders,ultimatelist)
len(listname_freq)

def takeSecond(elem):
    return int(elem[1])
listname_freq.sort(key=takeSecond)
#print(listname_freq)


In [157]:
def fixingnames(regex,rightname,listofunames):
    ult = "\n".join(listofunames)
    error_names = re.findall(regex,ult)
    error_numbs = []
    for toop in listname_freq:
        if toop[0] in error_names:
            n = toop[1]
            error_numbs.append(n)
        else:
            continue
    totalerror_names = sum(error_numbs)


    rightname = [rightname,totalerror_names]
    rightnamet = tuple(rightname)

    for toop in listname_freq:
        if toop[0] in error_names:
            listname_freq.remove(toop)
    listname_freq.append(rightnamet)
    listname_freq.sort(key=takeSecond)
    #print(listname_freq)
    
def fixallnames(listofunames):
    fixingnames("Smith, L.*ne","Shekter Smith, Liane",listofunames)
    fixingnames("W.+rfel,.B.*", "Wurfel, Brad",listofunames)
    fixingnames("W.+rfel,.S.*", "Wurfel, Sara",listofunames)
    fixingnames("Fonger.*", "Fonger, Ronald",listofunames)
    fixingnames("Edw.*", "Edwards, Marc",listofunames)
    fixingnames("Benz.*", "Benzie, Richard",listofunames)
    #fixingnames(regex,correctname,listofunames)

fixallnames(ultimatelist_senders)
print(listname_freq)
#be sure to enter prev cell before re-entering this cell

[('Hollins, Harvey', 1), ('Smith, Leslie', 1), ('Lyon, Nick', 1), ('Neeley, Sheldon', 1), ('Elliott, Larry', 1), ('Behalf, On', 1), ('Cline, Richard', 1), ('Washington, Valdemar', 1), ('Kosloski, Jessica', 1), ('Deltoral, Migue', 1), ('Bleicher, Matthew', 1), ('Paciorek, Josh', 1), ('Averill, James', 1), ('Lundy, Anastasia', 1), ('Williams, Felicia', 1), ('Boven, Jacquelyn', 1), ('Ploehn, Kelly', 1), ('Tetzlaff, Katherine', 1), ('Robeznieks, Andis', 1), ('Kennedy, Jordan', 1), ('Ockert, Katherine', 1), ('Seratt, Donna', 1), ('Forbes, Kevin', 1), ('Vaughn, Kari', 1), ('Wolf, Jennifer', 1), ('Michigan, Progress', 1), ('Gutting, Karen', 1), ('Hart, Nancy', 1), ('Biolchini, Amy', 1), ('epa,\nSchock', 1), ('Hill, Bryan', 1), ('Attorney, Michigan', 1), ('LeBlanc, Elizabeth', 1), ('Services, Human', 1), ('Kuhl, Richard', 1), ('Amin, Sameen', 1), ('Scott, Jackie', 1), ('Fitzner, Wendy', 1), ('Lesniewski, John', 1), ('Hedman, Of', 1), ('VanSickle, Michele', 1), ('Dupnack, Jessica', 1), ('Allema

In [158]:
len(listname_freq)

202

In [159]:
# # # SUBJECTS # # #

In [176]:
subjs = re.findall("Subject:.*[A-Za-z].*", emailstext)
fullsubjstr = '\n'.join(subjs)
subjs1 = re.sub("[FfWwRr]+[EeWwDd]:","",fullsubjstr)
subjs2 = re.findall("Subject:[\s]+(.*)",subjs1)
modifiedsubjstr = '\n'.join(subjs2)
print(modifiedsubjstr)
#print(fullsubjstr)

Flint Subsidy Projections
City of Flint DWRF Application Part 1
City of Flint DIWRF Application Part 1
City of Flint DWRF Application Part 1
Phosphate Fact Sheet
Request for Disclosure of Official Files - FOIA 0367-16
Flint water question CORRECTION
Flint water question
Flint water question CORRECTION
Flint water question CORRECTION
Flint water question CORRECTION
Eisenhower piece
Eisenhower piece
NEWS -- DEQ releases latest results from Flint schools water testing
NEWS: Latest updates from DEQ water testing at Flint Schools
NEWS: Latest updates from DEQ water testing at Flint Schools
NEWS: Latest updates from DEQ water testing at Flint Schools
NEWS: Latest updates from DEQ water testing at Flint Schools
Draft Background, Timeline, Quadrant Diagram
Phosphate Factsheet - NEW draft
Phosphate Factsheet - NEW draft
Phosphate Factsheet - NEW draft
Old Q&As
Eisenhower
DRAFT Flint Timeline ... and stuff
Elint Lead Monitoring Letter
FOR REVIEW: MDEQ - Flint Call Notes
FOR REVIEW: MDEQ - Flint 

In [172]:
len(re.findall("[Rr][Ee]:",fullsubjstr))

864

In [179]:
len(re.findall("[FfWw]+[WwDd]:",emailstext))

472

In [183]:
re.findall("Bcc",emailstext)

[]

In [187]:
len(subjs)

1975